In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import warnings
warnings.filterwarnings("ignore")
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/health-insurance-cross-sell-prediction/train.csv")
df_test = pd.read_csv("/kaggle/input/health-insurance-cross-sell-prediction/test.csv")

In [ ]:
df_test.head()

In [ ]:
df.info()

In [ ]:
print(df["Vehicle_Age"].unique())
print(df["Gender"].unique())
print(df["Vehicle_Damage"].unique())


- we do not need the id of the client for this study so we will delete the column
- we will also change the value of the colums [ Vehicle_Damage,Gender,Vehicle_Age] to numerical values

In [ ]:
def preprocessing(df):
    df.drop("id" , axis = 1 , inplace = True)
    df["Vehicle_Age"].replace({"> 2 Years":3,"1-2 Year":2,"< 1 Year":1},inplace = True)
    df["Gender"] = df["Gender"].map({"Male":1,"Female":0})
    df["Vehicle_Damage"] = df["Vehicle_Damage"].map({"Yes":1,"No":0})
    df["vintage"] = df["Vintage"].map(lambda x : 
                                   3 if x > 200 
                                   else 2 if x>100
                                   else 1
                                 )
    df = df.drop("Vintage",axis = 1)
    return df

In [ ]:
df= preprocessing(df)
df.head()

In [ ]:
df_test= preprocessing(df_test)
df_test.head()

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
#let's check if we have null values

In [ ]:
df.isnull().sum()

In [ ]:
#response population
df["Response"].value_counts()

In [ ]:
plt.title("Response population subscribers")
sns.countplot(df["Response"])
plt.show()

the amount of acceptance is very low and the data is imbalanced


In [ ]:
plt.title("subscribers Age population")
sns.distplot(df["Age"])
plt.show()

In [ ]:
plt.title("subscribers Age population ")
sns.boxplot(df["Age"],orient="v")
plt.show()

In [ ]:
sns.boxplot(x="Response",y ="Age",data = df)

- from analyzing the age population we can say that 75% of the subscribers have less or equal to 50 year
- people tend to accept Vehicle Insurance when the are much older

In [ ]:
plt.title("Gender population subscribers")
sns.countplot(df["Gender"])
plt.show()



the Gender data is slightly unbalanced we have  31069 males more in a 381109 subscribers

In [ ]:
plt.title("Gender population subscribers")
sns.countplot(x = "Gender" , hue= "Response",data = df)
plt.show()


the gender population data is balanced
but the gender does not give a good insight about the acceptance of the new Insurance 
the amount of Males accepting the new insurance more than females
the amount of Males refusing also the insurance new more than females 
so maybe we have this result because the total amount of male subscribers is more than female subscribers


In [ ]:
df["Driving_License"].value_counts()

In [ ]:
plt.title("Driving_License population subscribers")
sns.countplot(x = "Driving_License",data = df)
plt.show()

basically all the subscribers of the insurance have a driving licence


In [ ]:
plt.title("Driving_License/Response population subscribers/")
sns.countplot(x = "Driving_License" , hue= "Response",data = df)
plt.show()

it normal that just the subscribers with a driving license accept a driving insurance

In [ ]:
plt.figure(figsize=(30,30))
plt.title("Region_Code/Response population subscribers/")
sns.countplot(x = "Region_Code" ,hue = "Response" , data = df)
plt.show()

the company have more subscribers in the 28 region and less in the 51 region
we found the biger amount of positive reponses in the 28 which is correclated with the amount of the subscribers in the region


In [ ]:
plt.title("Previously_Insured/Response population subscribers/")
sns.countplot(x = "Previously_Insured" ,hue = "Response" , data = df)
plt.show()

basically only the subscribers without a vehicle insurance accept one

In [ ]:
plt.title("vehicle age/Response population subscribers")
sns.countplot(x = "Vehicle_Age" ,hue = "Response" , data = df)
plt.show()

In [ ]:
plt.title("Vehicle Damage/Response population subscribers")
sns.countplot(x = "Vehicle_Damage" ,hue = "Response" , data = df)
plt.show()

the subscribers that had a vehicle damage in the past tend to accept
the vehicle insurance more


In [ ]:
df.columns

In [ ]:
df["Annual_Premium"].median()

In [ ]:
plt.figure(figsize=(7,7))
plt.title("Annual Premium population")
sns.distplot(df["Annual_Premium"])
plt.show()

In [ ]:
plt.figure(figsize=(7,7))
sns.boxplot(x="Response",y ="Annual_Premium",data = df)
plt.show()

the Annual Premium data population is skewed 
with a median of 31669.0

In [ ]:
plt.figure(figsize=(10,10))
plt.title("Policy_Sales_Channel/Response population subscribers")
sns.countplot(y = "Policy_Sales_Channel" ,hue = "Response" , data = df.head(50))
plt.show()

In [ ]:
df[df["Response"] == 1]["Policy_Sales_Channel"].value_counts().to_frame().head(5)

the 5 best sales strategies to have more vehicle insurance subscribers are : 
[26,124,152,156,157]


In [ ]:
df["Policy_Sales_Channel"].value_counts().to_frame().head(5)

* the 5 best sales strategies to have more subscribers are : 
[152,26,124,160,156]

In [ ]:
sns.displot(df["vintage"])

In [ ]:

plt.title("vintage /Response population subscribers")
sns.countplot(x = "vintage" ,hue = "Response" , data = df)
plt.show()

been an old subscriber does not impact directly the subscribtion to the vechile insurance

In [ ]:
df.corr()

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(df.corr() , annot = True)

We can see that:
* The positive response is negatively correlated to Previously_Insured which is logical if we have insurance we will not accept 
* the positive response is slightly postivly correlated to Vehicle_Age and Vehicle_damage
* the positive response is slightly postivly correlated to the age which is what we found. Older subscribes tend to accept the vehicle insurance

In [ ]:
#Machine learning part 


In [ ]:
Y = df["Response"]
X = df.drop("Response",axis = 1)


In [ ]:
X.head()

In [ ]:
df_test.head()

In [ ]:
from sklearn.model_selection._split import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X,Y,test_size = 0.3)

In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

n_estimators = 10
max_depth = 3

Logistic_Regression = LogisticRegression(solver="liblinear",class_weight={1:2}, random_state=10)
Logistic_Regression.fit(X_train, y_train)

random_forest = RandomForestClassifier(
    n_estimators=n_estimators, max_depth=max_depth, random_state=10)
random_forest.fit(X_train, y_train)

gradient_boosting = GradientBoostingClassifier(
    n_estimators=n_estimators, max_depth=max_depth, random_state=10)
_ = gradient_boosting.fit(X_train, y_train)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import plot_roc_curve

fig, ax = plt.subplots()

models = [
    ("RF", random_forest),
    ("LR", Logistic_Regression),
    ("GBDT", gradient_boosting),
]

model_displays = {}
for name, model in models:
    model_displays[name] = plot_roc_curve(
        model, X_test, y_test, ax=ax, name=name)
_ = ax.set_title('ROC curve')

In [ ]:
""" we will choose the logistic Regression for the moment because we can
deal with the weight of the  labels  directly but we need to use a trick to the 
solve severe unbalanced labels """

Logistic_Regression = LogisticRegression(solver="liblinear",class_weight={1:2}, random_state=10)
Logistic_Regression.fit(X, Y)

In [ ]:
Logistic_Regression.score(X, Y)

In [ ]:
sub = pd.read_csv("/kaggle/input/health-insurance-cross-sell-prediction/sample_submission.csv")

sub["Response"] = Logistic_Regression.predict(df_test)

In [ ]:
sub["Response"].unique()